# 네이버 영화 감성분석
유명한 영화들에 대해 네이버 영화에서 평점, 영화평을 수집한 데이터. 
- <영화 아이디, 텍스트, 평점>

In [1]:
movie_reviews = '/home/paulkim/workspace/python/Korean_NLP/data/sample_naver_movie_5000/merged_comments_tokenized.txt'
with open(movie_reviews, encoding='utf-8') as f:
    for _ in range(3):
        print(next(f).strip())

127449	불편 한 현실 을 그대로 남아 냈다 . ' 최선을 다한 ' 미 나가 행복 해진다 면 좋 겠다 .. .	10
127449	서영 의는 추격자 지 칼깨 없다	10
127449	서영희 왕팬 입니다 . 변요한 도 여기 서 제대로 된 연기 를 기대 해 봅니다 . 무 도리 , 내생애 가장 아름다운 일주일 때부터 팬이 었 습니다 . 시간 이 갈수록 더 아름다 워지 십니다 . 화이팅	10


In [2]:
def load_comments(fname):
    with open(fname, encoding='utf-8') as f:
        docs = [doc.strip().split('\t') for doc in f]
        
    # 행에서 index를 제외한 값들만 파싱함
    docs = [(doc[1], int(doc[2])) for doc in docs if len(doc) == 3]
    texts, scores = zip(*docs)
    return texts, scores

texts, scores = load_comments(movie_reviews)
len(texts), len(scores)

(9950162, 9950162)

In [3]:
texts[1]

'서영 의는 추격자 지 칼깨 없다'

In [4]:
scores[1]

10

In [5]:
from collections import Counter

word_counter = Counter((word for text in texts for word in text.split()))

from soynlp.utils import get_process_memory
print('used memory = %.3f Gb' % get_process_memory())

used memory = 4.054 Gb


In [6]:
%%time
sorted(word_counter.items(), key=lambda x:x[1], reverse=True)[:20]

CPU times: user 146 ms, sys: 117 µs, total: 146 ms
Wall time: 145 ms


[('..', 4406260),
 ('영화', 3935219),
 ('.', 3131964),
 ('이', 2210520),
 ('는', 1272905),
 ('가', 1258466),
 ('의', 1220518),
 ('도', 1213741),
 ('!!', 1178909),
 ('ㅋㅋ', 1017934),
 ('재밌', 991375),
 ('관람객', 968283),
 ('재미', 935862),
 ('너무', 918724),
 ('을', 819675),
 ('!', 817826),
 ('정말', 793001),
 ('한', 781066),
 ('에', 780170),
 ('은', 776586)]

In [7]:
import time

start_cell = time.time()
n_words_before_pruning = len(word_counter)

min_count = 50

# 사전에 존재하는 단어들만으로 파싱하는 작업을 수행
word_dictionary = {word:freq for word,freq in word_counter.items() if freq >= min_count}
n_words_after_pruning  = len(word_dictionary)

print('%d --> %d' % (n_words_before_pruning, n_words_after_pruning))
total_time = time.time() - start_cell
print("total time : %.3f secs"%total_time)

401150 --> 49275
total time : 0.030 secs


In [8]:
print(type(word_dictionary))

<class 'dict'>


In [9]:
# 평점의 수준별 집계를 확인함
for score, score_freq in Counter(scores).items():
    print("score = %d:(%d, %.3f perc)"%(score, score_freq, 100*score_freq/len(scores)))

score = 10:(5275855, 53.023 perc)
score = 9:(1044252, 10.495 perc)
score = 6:(378270, 3.802 perc)
score = 8:(903213, 9.077 perc)
score = 7:(558421, 5.612 perc)
score = 1:(1031900, 10.371 perc)
score = 5:(296873, 2.984 perc)
score = 3:(148111, 1.489 perc)
score = 4:(169417, 1.703 perc)
score = 2:(143850, 1.446 perc)


### 영화 평점에 대한 수준을 긍정과 부정으로 파싱하는 작업
- texts는 각 단어를 토크나이징하기 이전의 자료구조를 갖고 있음
- word_dictionary는 우리가 전처리한 단어들이 들어있는 사전임
---

min count cutting을 하다보니 희귀한 단어로만 이뤄진 문장도 존재할 수 있음. 이 경우에는 zero vector가 만들어질 것임. 이를 방지하기 위해 학습용 데이터를 따로 만들어보자

그리고 binary classification을 하기 위해 3점 이하의 영화평을 negative, 9점 이상의 영화평은 positive로 만든다

    words = [word for word in text.split() if word in word_dictionary]
    if not words:
        continue
        
위의 내용은 text를 split()한 다음, 각 단어가 word_dictionary(min_count >= 50인 단어 집합)에 등록되어 있는지 확인하는 내용. 만약 words가 empty리스트이면 학습데이터에 추가하지 않고 다음 text를 살펴봄

    train_label.append(1 if score > 8 else -1)
    
위 내용은 평점이 9 혹은 10점이면 1이라는 label을 설정하고 아니면 -1이라는 label을 train_label에 추가함. 4~8점의 영화평들은 아래의 구문에 의해 존재하지 않음

    if 4 <= score <= 8:
        continue
        
그리고 조금더 특이하게 train_texts를 만들었음. words는 list of str임. 그렇기 때문에 train_texts는 list of list of str임

In [10]:
train_texts = []
train_label = []

for text, score in zip(texts, scores):
    if 4 <= score <= 8:
        continue
        
    words = [word for word in text.split() if word in word_dictionary]
    if not words:
        continue
    
    train_texts.append(words)
    train_label.append(1 if score > 8 else -1)

print('train data: %d --> %d' % (len(texts), len(train_texts)))
for label, label_freq in Counter(train_label).items():
    print('label = %d: (%d, %.3f perc)' % (label, label_freq, 100*label_freq/len(train_label)))

train data: 9950162 --> 7637001
label = 1: (6314758, 82.686 perc)
label = -1: (1322243, 17.314 perc)


CountVectorizer를 이용하여 train_x, term frequency matrix를 만들자. 이미 train_texts는 토크나이징이 완료되어 있음. CounterVectorizer에서 lowercase=False로 설정하고, tokenizer를 lambda x:x로 두면 아무런 처리를 하지 않는 체로 train_text를 입력할 수 있음

sparse matrix에서 이용한 단어의 갯수와 word_dictionary의 갯수가 같은지 확인해보고 min_df, max_df를 설정하지 않았기 때문에 빈도수가 50이상인 모든 단어들을 이용하여 sparse matrix를 만들었음

### 감정을 분류하기 위한 분류기 생성
- sklearn의 CounterVectorizer사용

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda x:x, lowercase = False)
train_x = vectorizer.fit_transform(train_texts)
train_x.shape

(7637001, 49275)

In [12]:
#with open('vocablist', 'w', encoding='utf-8') as f:
#     for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1]):
#         f.write('%s\n' % word)
vocablist = [word for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]
vocablist[1:10]

['!!', '!"', "!'", '!(', '!)', '!*', '!,', '!-', '!.']

In [13]:
SAVE = False # True
if SAVE:
    import pickle
    with open('sentiment_x.pkl', 'wb') as f:
        pickle.dump(train_x, f)

    with open('sentiment_y.pkl', 'wb') as f:
        pickle.dump(train_label, f)
        
    with open('sentiment_vocab.pkl', 'wb') as f:
        pickle.dump(vocablist, f)

In [14]:
from sklearn.linear_model import LogisticRegression

logistic_l2 = LogisticRegression(C=0.1)
logistic_l2.fit(train_x, train_label)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
for idx in [10, 100, 1000, 10000]:
    print('text: %s\nlabel: %d\n' % (train_texts[idx], train_label[idx]))
    
    prob = logistic_l2.predict_proba(train_x[idx,:])[0]
    print('class prob: (negative= %.3f, positive= %.3f' 
          % tuple(prob))
    
    pred_label = logistic_l2.predict(train_x[idx,:])[0]
    print('class = %s' % 'positive' if pred_label == 1 else 'negative')
    print('\n%s\n' % ('-'*50))

text: ['어제', '시사회', '다녀', '왔어요', '.', '기대', '이상', '으로', '너무', '몰입', '해서', '봤네요', '.', '불편', '하지만', '낯설', '지', '않은', '주인공', '의', '이야기', '와,', '그', '이야기', '를', '추리', '하게', '만', '들어', '서', '지루', '할', '틈이', '없', '었어요', '.', '서영희', '씨', '엄청', '예쁘고', '신', '수원', '감독', '님', '다음', '영화', '도', '챙겨', '볼께', '욧!']
label: 1

class prob: (negative= 0.000, positive= 1.000
class = positive

--------------------------------------------------

text: ['관람객', '모르', '는게', '약', '이', '라면', '아는', '게', '조금', '더', '쓴', '약', '인걸', '생각', '하게', '하는', '영화']
label: 1

class prob: (negative= 0.026, positive= 0.974
class = positive

--------------------------------------------------

text: ['예전', '처럼', '무슨', '여우비', '처럼', '망', '하지', '마시길', '~', '^^']
label: 1

class prob: (negative= 0.624, positive= 0.376
negative

--------------------------------------------------

text: ['밑', '댓글', '우연히', '보고', '말하', '러옴', '딱', '생', '일도', '아니', '고', '하루', '차이', '인데', '당연히', '우연', '이죠']
label: 1

class prob: (negative= 0.075, positive= 

Logistric Regression의 계수들을 확인해보자
    
    LogisticRegression.coef_
    
바이너리 분류를 수행했기 때문에 (1 by n_vocab)의 ndarray임. 다중분류를 수행한다면 (n_class by n_vocab)의 coefficient matrix가 만들어짐

학습하는 방법은 위와 동일. LogisticRegression이 train_label의 unique label의 갯수를 확인한 뒤, multi-class classification을 수행함. 이후 알아서 softmax regression으로 바뀜

In [16]:
print(type(logistic_l2.coef_))
print(logistic_l2.coef_.shape)

<class 'numpy.ndarray'>
(1, 49275)


ndarray를 list로 풀어보자. (1 by n_vocab)이므로 coefficient[0]을 하면 positive class를 예측하는 각 단어들의 계수들을 확인할 수 있음

In [17]:
coefficients = logistic_l2.coef_.tolist()
coefficients[0][:5]

[0.5172399912555908,
 0.14317486880702116,
 -0.19936269084683247,
 -0.08508651928689176,
 0.28469442933190064]

Positive class에 가까운 단어들을 확인. coeffient의 enumerate를 sorting하는 작업을 수행

In [18]:
sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
sorted_coefficients[:5]

[(45018, 4.181464147941289),
 (9440, 3.0354952369748216),
 (49045, 2.9896928736468014),
 (36434, 2.908639401476413),
 (37441, 2.9062310514210123)]

긍정적인 영화평에서 자주 나오는 단어들을 최상위부터 출력해보자

In [19]:
for word, coef in sorted_coefficients:
    print("%s (%.3f)"%(vocablist[word], coef))

틈이 (4.181)
꿀잼 (3.035)
흠잡을 (2.990)
있구만 (2.909)
재밋어요 (2.906)
시키지 (2.902)
굿 (2.822)
또보고싶 (2.762)
가는줄 (2.751)
여운이 (2.746)
굳 (2.745)
잊을수 (2.739)
짱임 (2.683)
안아까 (2.589)
존잼 (2.587)
굿굿 (2.562)
먹먹 (2.534)
쵝오 (2.530)
굿~ (2.498)
Good (2.465)
짱! (2.463)
안아까운 (2.432)
잼씀 (2.423)
뗄수 (2.421)
굿! (2.419)
최고 (2.397)
완벽 (2.388)
유쾌 (2.350)
낮아서 (2.323)
이제서야 (2.320)
졸잼 (2.314)
♥ (2.303)
재밋엇어요 (2.271)
슬퍼요 (2.235)
있던데 (2.223)
따뜻 (2.216)
♡ (2.200)
굳굳 (2.161)
최곱니다 (2.158)
낮지 (2.153)
대박 (2.147)
멋져요 (2.143)
낮네 (2.133)
만으로도 (2.120)
놓칠 (2.113)
전율이 (2.107)
손색이 (2.088)
짱이에요 (2.078)
짱짱 (2.071)
할틈 (2.070)
울었 (2.070)
♥♥ (2.063)
군더더기 (2.062)
낮지? (2.051)
저평가 (2.047)
웰메이드 (2.043)
잼나 (2.036)
아깝지 (2.033)
슬펐어요 (2.033)
꼭보 (2.033)
못본게 (2.011)
좋네요 (2.010)
이정도면 (2.009)
알이즈웰 (2.006)
개잼 (1.994)
좋던데 (1.989)
깔끔 (1.972)
짱 (1.970)
질리지 (1.963)
잼나게 (1.961)
완벽한 (1.961)
짱인듯 (1.961)
슬펐 (1.959)
뭉클 (1.956)
전율 (1.953)
놓칠수 (1.923)
짱이 (1.920)
잼써 (1.918)
잊을 (1.916)
흥해라 (1.908)
말할것도 (1.908)
깎는 (1.901)
잼잇 (1.901)
강추 (1.886)
재밋 (1.879)
쏠쏠 (1.875)
아름다

고급스러운 (0.427)
거고 (0.427)
비열한 (0.427)
잼쓸것 (0.427)
지존이다 (0.427)
을수 (0.427)
보게되다니 (0.427)
신념을 (0.427)
호드 (0.427)
여! (0.427)
존경스럽다. (0.427)
디오오빠 (0.426)
쵝 (0.426)
메릴 (0.426)
연연 (0.426)
해용 (0.426)
이냐리투 (0.426)
빌리 (0.426)
이러한 (0.426)
저쩌고 (0.426)
로멘틱 (0.426)
수식어 (0.426)
백개 (0.426)
넘은 (0.426)
THE (0.426)
이여서 (0.426)
만땅! (0.426)
견자단 (0.425)
깜찍한 (0.425)
지껄이지 (0.425)
마술 (0.425)
다크나이트 (0.425)
살자 (0.425)
예승이 (0.425)
언니 (0.425)
>. (0.425)
녀석 (0.424)
악성 (0.424)
용으 (0.424)
한이 (0.424)
보구싶 (0.424)
고마워 (0.424)
3D로 (0.424)
새 (0.424)
두배 (0.424)
야옹 (0.424)
니깐 (0.424)
로움 (0.424)
괸찬 (0.424)
뮤즈 (0.424)
이정 (0.424)
있으니까 (0.423)
너네는 (0.423)
삶. (0.423)
되내요 (0.423)
터닝 (0.423)
Captain (0.423)
늦은 (0.423)
립스 (0.423)
꺼이꺼이 (0.423)
안질 (0.423)
공길 (0.423)
있을까 (0.423)
석현 (0.423)
열연 (0.422)
조용한 (0.422)
열개도 (0.422)
넘사 (0.422)
촤고 (0.422)
el (0.422)
일어나 (0.422)
끊이지 (0.422)
너의 (0.422)
답다. (0.422)
노동자 (0.422)
단순 (0.422)
믿는다 (0.422)
중인 (0.422)
였어요 (0.421)
알라뷰 (0.421)
진진해 (0.421)
쓸거 (0.421)
적이 (0.421)
조니 (0.421)
고맙다 (0.421)
쉬운 (0.4

특히 (0.214)
켄이치 (0.214)
됬어요 (0.214)
면 (0.214)
면~ (0.214)
똘똘 (0.214)
리! (0.214)
키운 (0.214)
로망이다 (0.214)
짱b (0.213)
비타민 (0.213)
주이디 (0.213)
갓 (0.213)
전장 (0.213)
이긴 (0.213)
스의 (0.213)
조미 (0.213)
또안 (0.213)
울분을 (0.213)
보든 (0.213)
불굴의 (0.213)
이여 (0.213)
호프 (0.213)
나은듯 (0.213)
라는 (0.213)
갖 (0.213)
뜻하 (0.213)
히치콕의 (0.213)
울까봐 (0.213)
외침 (0.213)
몇이나 (0.213)
츄천 (0.213)
나다! (0.213)
시포 (0.213)
메디브 (0.213)
어두운 (0.213)
잼날 (0.213)
기들 (0.212)
이럼? (0.212)
치즈소스 (0.212)
고1때 (0.212)
매혹적이다 (0.212)
니요 (0.212)
셀수 (0.212)
네영 (0.212)
12 (0.212)
타게 (0.212)
94 (0.212)
포즈 (0.212)
착잡 (0.212)
본지 (0.212)
박재범 (0.212)
1기 (0.212)
한국 (0.212)
주어서 (0.212)
항해 (0.212)
투사부 (0.212)
ㅠ. (0.212)
도라에 (0.212)
쥑여 (0.212)
만행을 (0.212)
송경원 (0.212)
라의 (0.212)
Top (0.212)
아날 (0.212)
때려부시는 (0.212)
단한 (0.212)
관? (0.212)
체고시다 (0.212)
글라도스 (0.212)
합니 (0.212)
착하 (0.212)
검증된 (0.212)
미나문방구 (0.212)
스티븐스필버그 (0.212)
난잼 (0.212)
에르 (0.212)
소화해내는 (0.212)
는디 (0.211)
편안해지는 (0.211)
true (0.211)
크는 (0.211)
왜이 (0.211)
최오 (0.211)
김재 (0.211)
노예가 (0.211)
주윤

황제 (0.188)
젤잼 (0.188)
생기면 (0.188)
♥♥♥♥♥♥♥♥♥ (0.188)
단골 (0.188)
군사정권 (0.188)
심정을 (0.188)
MAX (0.188)
만1 (0.188)
쓸수 (0.188)
절해의 (0.188)
박사님 (0.188)
甲이 (0.188)
잇으면 (0.188)
낳는다. (0.188)
삼켜 (0.188)
Remember (0.187)
젊어 (0.187)
고위 (0.187)
보~ (0.187)
3보단 (0.187)
떨쳐 (0.187)
넹 (0.187)
촌철살인 (0.187)
IT (0.187)
왕두 (0.187)
만발~ (0.187)
퀼이 (0.187)
주다니 (0.187)
고의적 (0.187)
하느냐 (0.187)
크가 (0.187)
들과 (0.187)
비중 (0.187)
템포 (0.187)
멀록 (0.187)
않아 (0.187)
신안군 (0.187)
맨인더다크 (0.187)
이며 (0.187)
.성 (0.187)
덮고도 (0.187)
MJ (0.187)
빌뇌브 (0.187)
로얄 (0.187)
선배님 (0.187)
법도 (0.187)
컬투 (0.187)
어리석은 (0.187)
했을 (0.187)
발랄하 (0.187)
주목하라 (0.187)
월에 (0.187)
단1초도 (0.187)
잡힌 (0.187)
질색인데 (0.187)
영광이다. (0.187)
보내는 (0.187)
지들 (0.187)
더해져 (0.187)
통찰력 (0.187)
더해진 (0.186)
씨와 (0.186)
하넹 (0.186)
복원 (0.186)
촬영 (0.186)
터커 (0.186)
독창적인 (0.186)
거예요 (0.186)
팍! (0.186)
개츠비의 (0.186)
fffff (0.186)
3도 (0.186)
스티븐시걸 (0.186)
줘요. (0.186)
한당 (0.186)
게됨 (0.186)
여유를 (0.186)
쩜니다 (0.186)
K. (0.186)
부어서 (0.186)
sbs (0.186)
국민 (0.186)
내면 (0.186)
떔에 (0.18

인하여 (0.120)
대어를 (0.120)
얼룩 (0.120)
살앙 (0.120)
마냥 (0.120)
눈부시 (0.120)
싶엇던 (0.120)
준에 (0.120)
책과는 (0.120)
동욱 (0.120)
한편 (0.120)
직구 (0.120)
과속 (0.120)
에프터 (0.120)
못넘 (0.120)
질것 (0.120)
자비를 (0.120)
아… (0.120)
랏! (0.120)
않군 (0.120)
뽜이 (0.120)
쬐금 (0.120)
이쁘게나옴 (0.120)
표한 (0.120)
거면 (0.120)
가서 (0.120)
장담하건데 (0.120)
조건 (0.120)
안드 (0.120)
나올까말까한 (0.120)
니다 (0.120)
끈질기 (0.120)
!^ (0.120)
러뷰 (0.120)
존조 (0.120)
1급 (0.120)
매출 (0.120)
도도한 (0.120)
밑으 (0.120)
승호오빠 (0.120)
화사 (0.120)
며~ (0.120)
근혜 (0.120)
면서 (0.120)
를~ (0.120)
살라 (0.120)
넘이뻐요 (0.120)
이된 (0.120)
역설적인 (0.120)
콕콕 (0.120)
"I (0.120)
루팡3세 (0.120)
똥꼬쇼 (0.120)
장가 (0.120)
숨돌릴 (0.120)
캬아~ (0.120)
꼽으 (0.120)
어흑 (0.120)
트로이 (0.120)
신예 (0.120)
신성록 (0.120)
꼽자면 (0.120)
고고싱~ (0.120)
벤위쇼 (0.120)
수위는 (0.120)
헤헷 (0.120)
군필 (0.120)
고비드 (0.120)
<- (0.120)
3는 (0.120)
떨렸어요 (0.120)
즐감했 (0.120)
ny (0.120)
초의 (0.120)
방울방울 (0.120)
범접 (0.120)
사알짝 (0.120)
스승이 (0.120)
휙 (0.120)
사느냐 (0.120)
유를 (0.120)
오베 (0.120)
우왓 (0.120)
난걸 (0.120)
박히 (0.120)
많았 (0.120)
닫 (0.120)

...

비밀스러운 (-0.019)
하층민 (-0.019)
도서관 (-0.019)
갠전 (-0.019)
거라 (-0.019)
피튀 (-0.019)
바닥에 (-0.019)
침. (-0.019)
굿? (-0.019)
경악을 (-0.019)
결말 (-0.019)
수영복 (-0.019)
하려구요 (-0.019)
지속적 (-0.019)
택하 (-0.019)
가시 (-0.019)
장현성 (-0.019)
최다니엘 (-0.019)
트이 (-0.019)
이더라. (-0.019)
지겨운감이 (-0.019)
이감 (-0.019)
도큰 (-0.019)
:3 (-0.019)
무너질때 (-0.019)
송은 (-0.019)
() (-0.019)
설때 (-0.019)
마귀 (-0.019)
앞과 (-0.019)
짱은 (-0.019)
아르헨 (-0.019)
첫편 (-0.019)
007 (-0.019)
ㅕㅇ (-0.019)
방은 (-0.019)
모의 (-0.019)
탕 (-0.019)
됐네요. (-0.019)
슈와 (-0.019)
rk (-0.019)
입 (-0.019)
연타석 (-0.019)
야해요. (-0.019)
찬송가 (-0.019)
절차 (-0.019)
바른 (-0.019)
삼사 (-0.019)
생겻다 (-0.019)
존제 (-0.019)
순애 (-0.019)
힘겨워 (-0.019)
때깔이 (-0.019)
이중생 (-0.019)
뚫린 (-0.019)
물갈이 (-0.019)
범행을 (-0.019)
워쇼스키 (-0.019)
털보 (-0.019)
어딨냐 (-0.019)
홀과 (-0.019)
폭염 (-0.019)
미인이 (-0.019)
쯤? (-0.019)
망령 (-0.019)
뻔~ (-0.019)
배, (-0.019)
커피 (-0.019)
드라 (-0.019)
고와 (-0.019)
본뒤 (-0.019)
가비 (-0.019)
장이모 (-0.020)
십자가 (-0.020)
서면 (-0.020)
래드 (-0.020)
급인듯 (-0.020)
애정행각 (-0.020)
포인 (-0.020)
항거 (-0.020

복순 (-0.061)
써진 (-0.061)
되려나? (-0.061)
한인 (-0.061)
정형화된 (-0.061)
하더 (-0.061)
주라기공원 (-0.061)
괴기 (-0.061)
친해지고 (-0.061)
찡긋 (-0.061)
로이스 (-0.061)
왓어 (-0.061)
바나 (-0.061)
권해 (-0.061)
크업 (-0.061)
지났 (-0.061)
보소 (-0.061)
뿌지 (-0.061)
4랑 (-0.061)
넘의 (-0.061)
불행하게 (-0.061)
티는 (-0.061)
1월1일 (-0.061)
황사 (-0.061)
허풍이 (-0.061)
됐나? (-0.061)
서술 (-0.061)
코만도 (-0.061)
요령 (-0.061)
창주 (-0.061)
라던가 (-0.061)
할정 (-0.061)
앨리 (-0.061)
상기시켜 (-0.061)
듯싶 (-0.061)
집가 (-0.061)
합해 (-0.061)
힐튼 (-0.061)
구제 (-0.061)
뭐, (-0.061)
국인 (-0.061)
작사 (-0.061)
벌인 (-0.061)
형부 (-0.061)
율을 (-0.061)
공통 (-0.061)
계연성 (-0.061)
했더니 (-0.061)
다이어리 (-0.061)
알앗음 (-0.061)
여럿이 (-0.061)
마왕 (-0.061)
남쪽 (-0.061)
찾길 (-0.062)
입증 (-0.062)
잇을까 (-0.062)
뽑기 (-0.062)
넘쳐나는 (-0.062)
유년 (-0.062)
X이 (-0.062)
찍다 (-0.062)
LO (-0.062)
쳐진다. (-0.062)
컷는지 (-0.062)
혼재 (-0.062)
터라 (-0.062)
영향력이 (-0.062)
큐를 (-0.062)
써가며 (-0.062)
김무열 (-0.062)
삼동이 (-0.062)
정력 (-0.062)
노랫말 (-0.062)
용산에서 (-0.062)
개라 (-0.062)
지성이 (-0.062)
자아성찰 (-0.062)
두둥! (-0.062)
남? (-0.062)
많더 (-0.0

영은 (-0.120)
잼도 (-0.120)
하듯 (-0.120)
떄가 (-0.120)
병원에 (-0.120)
받거나 (-0.120)
매는 (-0.120)
요원들 (-0.120)
마십시오. (-0.120)
인터 (-0.120)
ff (-0.120)
러고 (-0.120)
쳐야지 (-0.120)
벗기는 (-0.120)
은참 (-0.120)
카우보이 (-0.120)
쌔 (-0.120)
대피 (-0.120)
머의 (-0.120)
갈대 (-0.120)
무상 (-0.120)
예? (-0.120)
한결같 (-0.120)
gp (-0.120)
짤려 (-0.120)
신공 (-0.121)
모래시계 (-0.121)
시로 (-0.121)
신은 (-0.121)
악어 (-0.121)
머. (-0.121)
로군 (-0.121)
그저그랬다. (-0.121)
멈 (-0.121)
술주정 (-0.121)
O임 (-0.121)
다분 (-0.121)
박살내 (-0.121)
발생한 (-0.121)
김조광수 (-0.121)
법적으로 (-0.121)
~씬 (-0.121)
건너지마오 (-0.121)
포는 (-0.121)
뭐고간에 (-0.121)
작다 (-0.121)
투하 (-0.121)
.에 (-0.121)
시니 (-0.121)
탄도 (-0.121)
'0' (-0.121)
헤픈 (-0.121)
무미 (-0.121)
네임밸류 (-0.121)
오용 (-0.121)
< (-0.121)
행동 (-0.121)
뒷북 (-0.121)
했니? (-0.121)
야~ (-0.121)
북극곰 (-0.121)
무색하게 (-0.121)
집어치워 (-0.121)
범한 (-0.121)
!내 (-0.121)
헐헐헐 (-0.121)
장님 (-0.121)
투척 (-0.121)
같네요. (-0.121)
애가 (-0.121)
무분별한 (-0.121)
없눈 (-0.121)
’ (-0.121)
워쇼 (-0.121)
뮤직비 (-0.121)
통탄할 (-0.121)
다를까? (-0.121)
단역이 (-0.121)
질이 (-0.121)
친구 (-0.1

너는내 (-0.210)
수술 (-0.210)
모친 (-0.210)
-0- (-0.210)
강신일 (-0.210)
합쳐놓은 (-0.210)
‥‥ (-0.210)
키스 (-0.210)
CE (-0.210)
받고 (-0.210)
봐? (-0.210)
카페에서 (-0.210)
침해 (-0.210)
복불복 (-0.210)
빨이 (-0.210)
훤히 (-0.210)
당하 (-0.210)
(- (-0.210)
대의 (-0.210)
ver (-0.210)
쩝! (-0.210)
아구 (-0.211)
화내 (-0.211)
갚은 (-0.211)
클라이막스 (-0.211)
풍 (-0.211)
기생 (-0.211)
모기 (-0.211)
얘네들 (-0.211)
돌입 (-0.211)
쫓아 (-0.211)
경악 (-0.211)
이태원 (-0.211)
웃통 (-0.211)
쓴건 (-0.211)
적중 (-0.211)
개운치 (-0.211)
날릴 (-0.211)
천원도 (-0.211)
성 (-0.211)
자화자찬 (-0.211)
생겼어 (-0.211)
훔쳐 (-0.211)
였으나 (-0.211)
자국의 (-0.211)
부르는 (-0.211)
설경구 (-0.211)
못보 (-0.211)
자작극 (-0.211)
따온 (-0.211)
내츄럴 (-0.211)
,,,,,,,,,, (-0.211)
믿지마라. (-0.211)
계실 (-0.211)
원수로 (-0.211)
지으면 (-0.212)
하늘나라로 (-0.212)
명이 (-0.212)
의뢰 (-0.212)
모바일로 (-0.212)
줬냐 (-0.212)
표 (-0.212)
뜬끔 (-0.212)
이름값 (-0.212)
신경쓰 (-0.212)
깨라 (-0.212)
도모 (-0.212)
먼데 (-0.212)
어린이들이 (-0.212)
뻣뻣 (-0.212)
난다. (-0.212)
랩 (-0.212)
5만명 (-0.212)
도끼 (-0.212)
더보 (-0.212)
든지 (-0.212)
애초부터 (-0.212)
리그 (-0.212)
부메랑 (-0.212)
받게 

다야 (-0.425)
둔갑시 (-0.425)
인민 (-0.425)
말자 (-0.425)
표본 (-0.425)
지겹도록 (-0.426)
개판이다. (-0.426)
반정부 (-0.426)
가관이 (-0.426)
대면 (-0.426)
중위 (-0.426)
미취학 (-0.426)
미취학아동 (-0.426)
담에 (-0.426)
재방 (-0.426)
갑수 (-0.426)
이씨 (-0.426)
충동 (-0.426)
왜바 (-0.427)
슈쥬 (-0.427)
곽 (-0.427)
설계 (-0.427)
우끼끼 (-0.427)
웩~ (-0.427)
정신차리세요 (-0.427)
나락 (-0.427)
학생들 (-0.427)
혈액형 (-0.427)
주려 (-0.427)
높을까 (-0.428)
지저분하 (-0.429)
없습 (-0.429)
예전 (-0.429)
6- (-0.429)
도나 (-0.429)
유분수지 (-0.429)
밥이 (-0.429)
카악 (-0.429)
좋지 (-0.429)
.뭔 (-0.430)
심했 (-0.430)
육두문자 (-0.430)
ㅂ2 (-0.430)
으이그 (-0.430)
급전 (-0.430)
는척 (-0.430)
흠 (-0.431)
곤 (-0.431)
머임? (-0.431)
분 (-0.431)
노린거 (-0.431)
아류작이 (-0.431)
믿는 (-0.431)
우매한 (-0.431)
사절 (-0.431)
부끄럽다. (-0.431)
믿으 (-0.431)
러워 (-0.431)
매치가 (-0.431)
극단적으로 (-0.432)
독수리 (-0.432)
라구 (-0.432)
창문 (-0.432)
느린 (-0.432)
그외 (-0.432)
인가? (-0.432)
망햇 (-0.432)
홍수 (-0.432)
김원희 (-0.432)
줬을 (-0.432)
8명 (-0.432)
써글 (-0.433)
내립니다. (-0.433)
연습좀 (-0.433)
좌좀 (-0.433)
못봐주겠더라 (-0.433)
지마 (-0.433)
누워 (-0.433)
이냐 (-0.433)
갔나요 (-

부정적인 영화평에서 자주 나오는 단어들을 상위 50개 출력해보자

In [20]:
for word, coef in sorted_coefficients[-50:]:
    print('%s (%.3f)' % (vocablist[word], coef))

거품이 (-2.310)
반개도 (-2.335)
나가고 (-2.347)
헛웃음 (-2.365)
형편없 (-2.375)
차라리 (-2.380)
김일병 (-2.398)
O기 (-2.401)
아깝다. (-2.407)
비추 (-2.409)
짜집기 (-2.420)
실망 (-2.422)
시계만 (-2.435)
망작 (-2.450)
잤 (-2.459)
잤다 (-2.463)
꽝 (-2.480)
아까 (-2.496)
엉망 (-2.499)
이딴 (-2.499)
낚였다 (-2.500)
하품만 (-2.504)
이것보단 (-2.512)
긴급조치19호 (-2.547)
거르는 (-2.551)
딴걸 (-2.558)
잤어요 (-2.573)
낚였 (-2.588)
별루 (-2.597)
2.22 (-2.619)
퇴보 (-2.642)
수면제 (-2.669)
졸려 (-2.741)
ㅁㅈㅎ (-2.762)
숙면 (-2.774)
하품 (-2.794)
돈아깝다 (-2.870)
졸작 (-2.953)
이딴게 (-2.964)
불면증 (-3.058)
과대평가 (-3.112)
방어율 (-3.125)
빵점 (-3.159)
최악이다. (-3.165)
내돈 (-3.248)
노잼 (-3.391)
핵노잼 (-3.700)
돈아까 (-3.792)
최악 (-4.055)
최악의 (-4.102)
